# Google Cloud の生成 AI サービス（言語モデル）で英文を日本語に要約して理解するサンプル

Vertex AI、および、Translation API のクライアントライブラリをインストールします。

In [1]:
!pip install google-cloud-aiplatform --upgrade --user
!pip install google-cloud-translate==2.0.1 --user

  Using cached google_cloud_core-2.3.3-py2.py3-none-any.whl (29 kB)
  Attempting uninstall: google-cloud-core
    Found existing installation: google-cloud-core 1.7.3
    Uninstalling google-cloud-core-1.7.3:
      Successfully uninstalled google-cloud-core-1.7.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-translate 2.0.1 requires google-cloud-core<2.0dev,>=1.1.0, but you have google-cloud-core 2.3.3 which is incompatible.
  Using cached google_cloud_core-1.7.3-py2.py3-none-any.whl (28 kB)
  Attempting uninstall: google-cloud-core
    Found existing installation: google-cloud-core 2.3.3
    Uninstalling google-cloud-core-2.3.3:
      Successfully uninstalled google-cloud-core-2.3.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency c

ここで一度、ランタイムのカーネルを再起動します。「Kernel」→「Restart Kernel」を選択してください。

Vertex AI のクライアントモジュールをインポートして、言語モデルのクライアントオブジェクトを生成します。

In [2]:
from vertexai.preview.language_models import TextGenerationModel
generation_model = TextGenerationModel.from_pretrained('text-bison@001')

量子コンピューターを説明した英文を短く要約するように依頼するプロンプトを用意します。

In [3]:
prompt = """
Provide a very short summary, no more than three sentences, for the following article:

Our quantum computers work by manipulating qubits in an orchestrated fashion that we call quantum algorithms.
The challenge is that qubits are so sensitive that even stray light can cause calculation errors — and the problem worsens as quantum computers grow.
This has significant consequences, since the best quantum algorithms that we know for running useful applications require the error rates of our qubits to be far lower than we have today.
To bridge this gap, we will need quantum error correction.
Quantum error correction protects information by encoding it across multiple physical qubits to form a “logical qubit,” and is believed to be the only way to produce a large-scale quantum computer with error rates low enough for useful calculations.
Instead of computing on the individual qubits themselves, we will then compute on logical qubits. By encoding larger numbers of physical qubits on our quantum processor into one logical qubit, we hope to reduce the error rates to enable useful quantum algorithms.

Summary:
"""

言語モデルの API を呼び出して応答を取得します。

In [4]:
response = generation_model.predict(
        prompt, temperature=0.2, max_output_tokens=1024, top_k=40, top_p=0.8
    ).text

得られた応答を表示します。

In [5]:
print(response)

Quantum computers are very sensitive and errors can occur easily.
Quantum error correction protects information by encoding it across multiple physical qubits to form a “logical qubit”.
By encoding larger numbers of physical qubits on our quantum processor into one logical qubit, we hope to reduce the error rates to enable useful quantum algorithms.


英語の応答を日本語に翻訳するために、Translate API のモジュールをインポートして、クライアントオブジェクトを取得します。

In [6]:
from google.cloud import translate_v2 as translate
translate_client = translate.Client()

英語に翻訳可能な言語のリストを表示します。日本語は 'ja' で指定できることがわかります。

In [7]:
translate_client.get_languages(target_language='en')

[{'language': 'af', 'name': 'Afrikaans'},
 {'language': 'sq', 'name': 'Albanian'},
 {'language': 'am', 'name': 'Amharic'},
 {'language': 'ar', 'name': 'Arabic'},
 {'language': 'hy', 'name': 'Armenian'},
 {'language': 'as', 'name': 'Assamese'},
 {'language': 'ay', 'name': 'Aymara'},
 {'language': 'az', 'name': 'Azerbaijani'},
 {'language': 'bm', 'name': 'Bambara'},
 {'language': 'eu', 'name': 'Basque'},
 {'language': 'be', 'name': 'Belarusian'},
 {'language': 'bn', 'name': 'Bengali'},
 {'language': 'bho', 'name': 'Bhojpuri'},
 {'language': 'bs', 'name': 'Bosnian'},
 {'language': 'bg', 'name': 'Bulgarian'},
 {'language': 'ca', 'name': 'Catalan'},
 {'language': 'ceb', 'name': 'Cebuano'},
 {'language': 'ny', 'name': 'Chichewa'},
 {'language': 'zh', 'name': 'Chinese (Simplified)'},
 {'language': 'zh-TW', 'name': 'Chinese (Traditional)'},
 {'language': 'co', 'name': 'Corsican'},
 {'language': 'hr', 'name': 'Croatian'},
 {'language': 'cs', 'name': 'Czech'},
 {'language': 'da', 'name': 'Danish

日本語に翻訳する関数を定義します。

In [8]:
def translate_text(text):
    translated = []
    for line in text.split('\n'):
        result = translate_client.translate(line, target_language='ja')
        translated.append(result['translatedText'])
    return '\n'.join(translated)

定義した関数を用いて日本語に翻訳します。

In [9]:
print(translate_text(response))

量子コンピューターは非常に敏感であり、エラーが発生しやすいです。
量子エラー訂正は、複数の物理量子ビットにわたって情報をエンコードして「論理量子ビット」を形成することで情報を保護します。
量子プロセッサ上で多数の物理量子ビットを 1 つの論理量子ビットにエンコードすることで、エラー率を低減し、有用な量子アルゴリズムを実現できるようにしたいと考えています。


要約と翻訳をまとめて実施する関数を定義します。

In [10]:
def summarize_and_translate_text(text):
    prompt = """
Provide a very short summary, no more than three sentences, for the following article:

{}

Summary:
""".format(text)
    response = generation_model.predict(
        prompt, temperature=0.2, max_output_tokens=1024, top_k=40, top_p=0.8
    ).text
    return translate_text(response)

この関数は、要約対象の文章だけを入力すると、要約結果が得られます。

In [11]:
text = """
Our quantum computers work by manipulating qubits in an orchestrated fashion that we call quantum algorithms.
The challenge is that qubits are so sensitive that even stray light can cause calculation errors — and the problem worsens as quantum computers grow.
This has significant consequences, since the best quantum algorithms that we know for running useful applications require the error rates of our qubits to be far lower than we have today.
To bridge this gap, we will need quantum error correction.
Quantum error correction protects information by encoding it across multiple physical qubits to form a “logical qubit,” and is believed to be the only way to produce a large-scale quantum computer with error rates low enough for useful calculations.
Instead of computing on the individual qubits themselves, we will then compute on logical qubits. By encoding larger numbers of physical qubits on our quantum processor into one logical qubit, we hope to reduce the error rates to enable useful quantum algorithms.
"""

print(summarize_and_translate_text(text))

量子コンピューターは非常に敏感で、エラーが発生しやすくなっています。
量子エラー訂正は、複数の物理量子ビットにわたって情報をエンコードして「論理量子ビット」を形成することで情報を保護します。
量子プロセッサ上で多数の物理量子ビットを 1 つの論理量子ビットにエンコードすることで、エラー率を低減し、有用な量子アルゴリズムを実現できるようにしたいと考えています。
